In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 1

In [5]:
a = np.array([49,58,75,110,112,132,151,276,281,362])
h0=200

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

In [10]:
stats.wilcoxon(a - h0)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [12]:
round(stats.wilcoxon(a - h0)[1],4)

0.2845

# 2

In [13]:
les19 = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
les9 = np.array([17,18,18,15,12,4,14,15,10])

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [15]:
stats.mannwhitneyu(les9, les19, alternative='less')

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [16]:
round(stats.mannwhitneyu(les9, les19, alternative='less')[1],4)

0.029

# 3

In [19]:
df = pd.read_csv('challenger.txt', sep='\t')

In [20]:
df.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [37]:
df_ok = np.array(df[df['Incident'] == 0]['Temperature'])
df_br = np.array(df[df['Incident'] == 1]['Temperature'])

In [30]:
np.random.seed(0)

In [31]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [35]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [39]:
ok_mean_score = list(map(np.mean, get_bootstrap_samples(df_ok, 1000)))
br_mean_score = list(map(np.mean, get_bootstrap_samples(df_br, 1000)))

In [43]:
delta_mean_scores = list(map(lambda x: x[0] - x[1], zip(ok_mean_score, br_mean_score)))

In [44]:
print("95% confidence interval for the difference between medians",  stat_intervals(delta_mean_scores, 0.05))

95% confidence interval for the difference between medians [1.42299107 7.93861607]


In [46]:
round(stat_intervals(delta_mean_scores, 0.05)[0],4)

1.423

# 4

In [47]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [48]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [49]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), list(filter(lambda i: i not in index, range(n)))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [50]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [55]:
np.random.seed(0)
print("p-value: %f" % round(permutation_test(df_ok, df_br, max_permutations = 10000),4))

p-value: 0.007000
